In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Conv2DTranspose
from keras.layers import Dropout
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
#from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.core import Flatten
from keras.optimizers import Adam
import numpy as np
from PIL import Image
import os
import glob
import random

n_colors = 3

/mnt/nfs/sakai/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def generator_model():
    print("Generator's model")
    
    dim = 4
    depth = 1024 # 512?
    
    model = Sequential()
    
    model.add(Dense(dim*dim*depth, input_shape=(100,)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))
    
    # Denseを挟むべき？
    
    model.add(Reshape((dim,dim,depth)))
    model.add(Dropout(0.5))
    
    # 1層目
    model.add(Conv2DTranspose(int(depth/2), kernel_size=(4, 4), strides=(2, 2), padding='same', )) # 8*8
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))

    # 2層目
    model.add(Conv2DTranspose(int(depth/4), kernel_size=(4, 4), strides=(2, 2), padding='same', )) # 16*16
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))
    
    # 3層目
    #model.add(Dropout(0.5))
    
    model.add(Conv2DTranspose(int(depth/8), kernel_size=(4, 4), strides=(2, 2), padding='same', )) # 32*32
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))
    
    # 出力層
    model.add(Conv2DTranspose(n_colors, kernel_size=(4, 4), strides=(2, 2), padding='same', )) # 64*64
    # 出力にはBatchNormalizationを使わない，　不安定になるため
    model.add(Activation('tanh'))
    
    print(model.summary())

    return model

In [3]:
def discriminator_model():
    
    print("Discriminator's model")
    
    model = Sequential()
    
    # 1層目
    model.add(Conv2D(64, kernel_size=(4, 4), strides=(2, 2), input_shape=(64, 64, n_colors), padding='same'))
    #model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))
    
    # 3層目
    model.add(Conv2D(128, kernel_size=(4, 4), strides=(2, 2), padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))
    
    # 4層目
    model.add(Conv2D(256, kernel_size=(4, 4), strides=(2, 2), padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))
    
    # 5層目
    model.add(Conv2D(512, kernel_size=(4, 4), strides=(2, 2), padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))
    
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(LeakyReLU(0.2))
    
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    print(model.summary())
    
    return model

In [4]:
def generator_containing_discriminator(generator, discriminator):
    
    print("Combined model")
    
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    
    print(model.summary())
    
    return model

In [5]:
def image_batch(batch_size):
    
    #files = glob.glob("../animeface-character-dataset/img/*.png", recursive=True)
    files = glob.glob("../data/img_align_celeba/*.jpg", recursive=True)
    files = random.sample(files, batch_size)
    # print(files)
    res = []
    for path in files:
        img = Image.open(path)
        img = img.resize((64, 64))
        arr = np.array(img)
        arr = (arr - 127.5) / 127.5
        arr.resize((64, 64, n_colors))
        res.append(arr)
    return np.array(res)

In [6]:
def combine_images(generated_images, cols=5, rows=5):
    shape = generated_images.shape
    h = shape[1]
    w = shape[2]
    image = np.zeros((rows * h,  cols * w, n_colors))
    for index, img in enumerate(generated_images):
        if index >= cols * rows:
            break
        i = index // cols
        j = index % cols
        image[i*h:(i+1)*h, j*w:(j+1)*w, :] = img[:, :, :]
    image = image * 127.5 + 127.5
    image = Image.fromarray(image.astype(np.uint8))
    return image

In [7]:
def set_trainable(model, trainable):
    model.trainable = trainable
    for layer in model.layers:
        layer.trainable = trainable

In [8]:
def main():
    batch_size = 200
    discriminator = discriminator_model()
    generator = generator_model()

    discriminator_on_generator = generator_containing_discriminator(generator, discriminator)
    set_trainable(discriminator, False)
    discriminator_on_generator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))
    
    set_trainable(discriminator, True)
    discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.00001, beta_1=0.1)) # Dが強いので学習率は下げるべき？0.00001, 0.1
    
    for i in range(500 * 100):
        
        batch_images = image_batch(batch_size)
        
        noise = np.random.normal(size=[batch_size, 100], loc=0, scale=0.02) # ガウス分布から100次元のベクトル
        #noise = np.random.uniform(size=[batch_size, 100], low=-1.0, high=1.0)　# 一様分布
        generated_images = generator.predict(noise)
        #generated_images = np.concatenate(generated_images)
        
        #X = np.concatenate((batch_images, generated_images))　# 本物と偽物を結合
        #y = [1] * batch_size + [0] * batch_size
        #d_loss = discriminator.train_on_batch(X, y)
        
        d_loss_real = discriminator.train_on_batch(batch_images, np.ones((batch_size, 1)))
        d_loss_fake = discriminator.train_on_batch(generated_images, np.zeros((batch_size, 1)))
        
        noise = np.random.normal(size=[batch_size, 100], loc=0, scale=0.02) # ガウス分布から100次元のベクトル
        #noise = np.random.uniform(size=[batch_size, 100], low=-1.0, high=1.0)
        g_loss = discriminator_on_generator.train_on_batch(noise, np.ones((batch_size, 1))) # サチるから目的関数を変更
        
        if i % 500 == 0:
            print("step %d d_loss_real, d_loss_fake, g_loss : %g,\t %g,\t %g" % (i, d_loss_real, d_loss_fake, g_loss))
            image = combine_images(generated_images)
            #os.system('mkdir -p ./gen_images')
            image.save("./generated_img/gen%05d.png" % i)
            # generator.save_weights('generator.h5', True)
            # discriminator.save_weights('discriminator.h5', True)

In [ ]:
main()

Discriminator's model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        3136      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       131200    
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 128)       512       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 256)         524544    
_________________________________________________________________
batch_normalization_2 (Batch (None, 8, 8, 256)        

/mnt/nfs/sakai/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


step 0 d_loss_real, d_loss_fake, g_loss : 0.785613,	 0.695549,	 0.392375
step 100 d_loss_real, d_loss_fake, g_loss : 1.09891,	 2.58659,	 0.515683
step 200 d_loss_real, d_loss_fake, g_loss : 1.07538,	 1.46528,	 0.977968
step 300 d_loss_real, d_loss_fake, g_loss : 1.15772,	 0.946298,	 0.850252
step 400 d_loss_real, d_loss_fake, g_loss : 0.725339,	 0.46119,	 0.33599
step 500 d_loss_real, d_loss_fake, g_loss : 0.591018,	 0.592889,	 0.0833461
step 600 d_loss_real, d_loss_fake, g_loss : 1.04391,	 1.23012,	 0.577967
step 700 d_loss_real, d_loss_fake, g_loss : 0.882404,	 0.880936,	 0.780667
step 800 d_loss_real, d_loss_fake, g_loss : 0.763239,	 0.654256,	 0.616224
step 900 d_loss_real, d_loss_fake, g_loss : 0.742738,	 0.801623,	 0.865017
step 1000 d_loss_real, d_loss_fake, g_loss : 0.850873,	 0.819277,	 0.725398
step 1100 d_loss_real, d_loss_fake, g_loss : 0.827674,	 0.812343,	 0.650468
step 1200 d_loss_real, d_loss_fake, g_loss : 0.794654,	 0.815304,	 0.752202
step 1300 d_loss_real, d_loss_fa